Pipelines chains toghether multiple steps so that the output  of each step is used as input  to the next step 

Pipelines makes it easy to apply the same preprocessing to train and test.

In [3]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


In [4]:
df = pd.read_csv("train.csv")

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Drop unwanted column

In [10]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)


KeyError: "['PassengerId', 'Name', 'Ticket', 'Cabin'] not found in axis"

In [13]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [17]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

Apply imputation for missing data

In [21]:
# Filling values with mean
si_age = SimpleImputer()

# Filling values with  most frequent data set
si_embark = SimpleImputer(strategy='most_frequent') 

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embark = si_embark.fit_transform(X_train[['Embarked']])

X_test_age = si_age.fit_transform(X_test[['Age']])
X_test_embark = si_embark.fit_transform(X_test[['Embarked']])


In [25]:
# One hot encoding
ohe_sex = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embark = OneHotEncoder(sparse=False,handle_unknown='ignore')

X_train_sex =  ohe_sex.fit_transform(X_train[['Sex']])
X_train_embark =  ohe_embark.fit_transform(X_train_embark)

X_test_sex =  ohe_sex.fit_transform(X_test[['Sex']])
X_test_embark =  ohe_embark.fit_transform(X_test_embark)


Dropping the untouched column from X_train and X_test 

In [26]:
X_train_rem = X_train.drop(columns =["Sex","Age","Embarked"])
X_test_rem = X_test.drop(columns =["Sex","Age","Embarked"])

In [29]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embark),axis=1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embark),axis=1)

using Decision tree classifier predicting values

In [32]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

In [36]:
y_predict = clf.predict(X_test_transformed)

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
accuracy_score(y_test,y_predict)

0.7486033519553073

709    1
439    0
840    0
720    1
39     1
      ..
433    0
773    0
25     1
84     1
10     1
Name: Survived, Length: 179, dtype: int64